### Libraries and Files

In [109]:
import pandas as pd
import warnings
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [110]:
df = pd.read_csv('/Users/Michael/Documents/GitHub/ssicsync/Webscrap/Webscrape from Linkedin.csv')
ref = pd.read_csv('/Users/Michael/Documents/GitHub/ssicsync/Webscrap/List of 90 Coy and SSIC.csv', dtype={'ssic_code': str})
ssic = pd.read_excel('/Users/Michael/Documents/GitHub/ssicsync/ssic2020-detailed-definitions.xlsx', skiprows=[0,1,2,3], usecols=[0,1,3,5], dtype={'SSIC 2020': str})
pdf = pd.read_excel('C:/Users/Michael/Documents/GitHub/ssicsync/Webscrap/AR Report Data/extracted_notes_pages_with_uen.xlsx')

In [111]:
linkedInModelResults_path = '/Users/Michael/Documents/GitHub/ssicsync/recommendationModels/linkedInFullModelResults.xlsx'
linkedInModel_path = '/Users/Michael/Documents/GitHub/ssicsync/recommendationModels/linkedInModel'

pdfModelResults_path = '/Users/Michael/Documents/GitHub/ssicsync/recommendationModels/pdfFullModelResults.xlsx'
pdfModel_path = '/Users/Michael/Documents/GitHub/ssicsync/recommendationModels/pdfModel'

linkedInPdfModelResults_path = '/Users/Michael/Documents/GitHub/ssicsync/recommendationModels/linkedInPdfFullModelResults.xlsx'
linkedInPdfModel_path = '/Users/Michael/Documents/GitHub/ssicsync/recommendationModels/linkedInPdfModel'

### Recommender Model Function

In [130]:
def recommendationModel(df = None, ssic = None, withSSIC = True, epochs = 40, vector_size = 64, top = 5, trainPercentage_0to1 = 0.7):

    df['ssic_code'] = df['ssic_code'].astype(str).str.zfill(5)
    df.reset_index(drop = True, inplace = True)

    # 70-30 Train-Test split (randomised selection):
    train = df.sample(round(df.shape[0]*trainPercentage_0to1)).reset_index(drop = True)
    test = df[~df.Company.isin(train.Company)].reset_index(drop = True)

    if withSSIC == True:
        # NOTE include line below if want to add original SSIC text reference into training data:
        ssic.reset_index(drop = True, inplace = True)
        train = pd.concat([train, ssic], axis = 0).reset_index(drop = True)

    # Model training
    textonly = train['Text Content']
    text_tokens = [word_tokenize(t.lower()) for t in textonly]
    tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(text_tokens)]
    model = Doc2Vec(vector_size=vector_size, min_count=2, epochs=epochs)
    model.build_vocab(tagged_data)
    model.train(tagged_data, total_examples=model.corpus_count, epochs=epochs)

    # Parsing model outputs into a df (with dups)
    recsys_df = pd.DataFrame(columns = ["Company", "actualSSIC", "recSSIC", "score", "actualTextContent", "recommendedTextContent"])
    for index, series in test.iterrows():
        test_doc = word_tokenize(series['Text Content'].lower())
        test_vec = model.infer_vector(test_doc)
        results = model.docvecs.most_similar(positive=[test_vec], topn=top)
        for i, (index, similarity_score) in enumerate(results):
            coy = series["Company"]
            actualssic = series['ssic_code']
            recssic = train["ssic_code"][index]
            actualTextContent = series["Text Content"]
            recommendedTextContent = train["Text Content"][index]
            score = similarity_score
            row = [coy, actualssic, recssic, score, actualTextContent, recommendedTextContent]
            recsys_df.loc[len(recsys_df)] = row
    recsys_df['rank'] = recsys_df.sort_values(by = ['Company', 'score'], ascending=[True, False]).groupby('Company').cumcount()+1
    recsysForValidation_df = recsys_df.copy()
    training_df = train.copy()
    testing_df = test.copy()

    recsys_df.drop(columns = ["actualTextContent", "recommendedTextContent"], inplace = True)
    recsys_df['actualSSIC'] = recsys_df['actualSSIC'].apply(lambda x: x[0:2])
    recsys_df['recSSIC'] = recsys_df['recSSIC'].apply(lambda x: x[0:2])

    recsysForValidation_df['actualSSICDescription'] = ''
    recsysForValidation_df['recSSICDescription'] = ''
    for index, series in recsysForValidation_df.iterrows():
        ssicGrouped = ssic.groupby('ssic_code')['Text Content'].apply(lambda x: ' '.join(x)).reset_index()

        try:    
            actualSSICDescription = ssicGrouped[ssicGrouped.ssic_code == series.actualSSIC].reset_index(drop=True)['Text Content'].values[0]
            recsysForValidation_df.loc[index, 'actualSSICDescription'] = actualSSICDescription
        except Exception as e:
            print('actualSSICDescription:')
            print(ssicGrouped[ssicGrouped.ssic_code == series.actualSSIC].reset_index(drop=True)['Text Content'])
            print(f"Error: {e}")
            print(f"SSIC in Master List: {series.actualSSIC}")

        try:
            recSSICDescription = ssicGrouped[ssicGrouped.ssic_code == series.recSSIC].reset_index(drop=True)['Text Content'].values[0]
            recsysForValidation_df.loc[index, 'recSSICDescription'] = recSSICDescription
        except Exception as e:
            print('recSSICDescription:')
            print(ssicGrouped[ssicGrouped.ssic_code == series.actualSSIC].reset_index(drop=True)['Text Content'])
            print(f"Error: {e}")
            print(f"SSIC in Master List: {series.actualSSIC}")

    recsysForValidation_df = recsysForValidation_df[['Company', 'actualSSIC', 'recSSIC', 'score', 'rank', 'actualTextContent', 'recommendedTextContent', 'actualSSICDescription', 'recSSICDescription']]

    #########################################
    
    recsysAccuracy_df = recsys_df.copy()

    print('Model Results:')
    recsysAccuracy_df.loc[recsysAccuracy_df.actualSSIC == recsysAccuracy_df.recSSIC, 'match'] = 1
    recsysAccuracy_df.loc[recsysAccuracy_df.match != 1, 'match'] = 0
    recsysAccuracy_df = recsysAccuracy_df.sort_values(by = ['Company', 'match'], ascending=False).groupby('Company').head(1).reset_index(drop = True)
    matchAccuracy = len(recsysAccuracy_df[recsysAccuracy_df.match == 1].match)/recsysAccuracy_df.shape[0]
    print(f'Accuracy of Recommendation Model: {round(matchAccuracy*100,1)}%')

    print(f"Overall Similarity Score: {round(recsys_df.score.mean(),2)} out of 1.00")

    recsysStats_df = recsys_df.copy()
    recsysStats_df['recSSIC'] = recsysStats_df['recSSIC'].astype('int64')
    recsysStats_df = recsysStats_df.groupby('Company')['recSSIC'].agg(['mean', 'std']).reset_index()
    print(f"Overall Stats (99 Divisions in Total):\n{round(recsysStats_df.mean(),1)}")

    print_statements = {
    "Evaluation Metrics": ["Overall Accuracy", "Similarity Score (Average)", "Mean (99 Divisions in Total)", "Standard Deviation (99 Divisions in Total)"],
    "Values": [f"{matchAccuracy}", f"{recsys_df.score.mean()}", f"{recsysStats_df['mean'].mean()}", f"{recsysStats_df['std'].mean()}"]
    }
    modelResults_df = pd.DataFrame(print_statements)

    return recsys_df, recsysStats_df, recsysForValidation_df, training_df, testing_df, model, modelResults_df

In [113]:
for column in ssic.columns:
    ssic.loc[((ssic[column] == '<Blank>') | (ssic[column].isnull())), column] = ''
ssic = ssic[~ssic['SSIC 2020'].str.match(r'^[A-Z]$')]

# just predicting up to Division level in the SSIC code
# ssic['SSIC 2020'] = ssic['SSIC 2020'].apply(lambda x: str(x)[0:2])

# Define a cleaning function
def clean_text(text):
    if isinstance(text, str):
        # Replace newline characters with space
        text = text.replace('\n', ' ')
        # Replace special characters (bullet points) with a consistent delimiter
        text = text.replace('•', '- ')
        # Strip leading and trailing whitespace
        text = text.strip()
        # Replace multiple spaces with a single space
        text = ' '.join(text.split())
    return text

# Apply the cleaning function to the 'Info' column
ssic['Detailed Definitions'] = ssic['Detailed Definitions'].apply(clean_text)
ssic['Examples of Activities Classified Under this Code'] = ssic['Examples of Activities Classified Under this Code'].apply(clean_text)

ssic['textonly'] = ssic['SSIC 2020 Title'] + '.' + ssic['Detailed Definitions'] + '.' + ssic['Examples of Activities Classified Under this Code']
ssic = ssic[['SSIC 2020', 'textonly']]
ssic.columns = ['ssic_code', 'Text Content']
ssic = ssic[ssic['Text Content'].notnull()]

### LinkedIn

In [114]:
# Step 1: Remove leading and trailing whitespace
df['Text Content'] = df['Text Content'].str.strip()

# Step 2: Replace newline characters with spaces
df['Text Content'] = df['Text Content'].str.replace('\r\n', ' ', regex=False)
df['Text Content'] = df['Text Content'].str.replace('\n', ' ', regex=False)

# Step 3: Fill missing values with a placeholder
# df['Text Content'] = df['Text Content'].fillna('')

# Optional Step 4: Normalize text by converting to lowercase (if needed)
df['Text Content'] = df['Text Content'].str.lower()

In [115]:
ref.rename(columns = {'entity_name': 'Company'}, inplace = True)
ref = ref.sort_values(by = ['Company', 'ssic_code']).drop_duplicates(subset = 'Company', keep = 'first')
ref = ref[['Company', 'ssic_code', 'UEN']]

test = pd.merge(df, ref, how = 'left', on = 'Company')

In [116]:
# NOTE There's discrepency in PDFs vs 'List of 90 Coy and SSIC' company names (12 companies missing) ..
# .. [Most were resolved by putting a '.' to List of 90 Coy and SSIC's df]
ref['Company'] = ref['Company'].apply(lambda x: x[:-1] if x[-1] == '.' else x)

test2 = pd.merge(test[test.ssic_code.isnull()], ref, how = 'left', on = 'Company')
test2.drop(columns='ssic_code_x', inplace = True)
test2.rename(columns={'ssic_code_y':'ssic_code'}, inplace = True)

In [117]:
# NOTE Web scrapping codes is taking the first profile in the search page, which may be the wrong company. Hence, scrapped info may be wrong.
## ACCRELIST LTD linkedin info seems wrong, but pdf and List of 90 Coy and SSIC's df seems correct (misaligned names!) ..
## TODO Probably should webscrape using the names from List of 90 Coy and SSIC's df instead of pdf names (since it's the source of truth for SSIC codes)?
df = pd.concat([test[~ test.ssic_code.isnull()], test2], axis = 0)

# NOTE Out of 90 coys, ..
## .. 18 has no LinkedIn content, ..
## .. 1 has PDF but not in List of 90 Coy and SSIC's df (ACMA LTD), ..
## .. 1 has PDF and in List of 90 Coy and SSIC's df (ACMA LTD) but partially different name (ACCRELIST LTD) [hence can't match to get SSIC codes!]
df = df[['Company', 'ssic_code', 'Text Content']][(df.ssic_code.notnull()) & (df['Text Content'].notnull())].reset_index(drop = True)

In [118]:
accuracy = 0
for i in range(1,11):
# while accuracy < 0.55:
    recsys_df, recsysStats_df, recsysForValidation_df, training_df, testing_df, model, modelResults_df = recommendationModel(df = df, ssic = ssic,
                                                                                                    withSSIC = True, epochs = 40, vector_size = 64,
                                                                                                    top = 5, trainPercentage_0to1 = 0.7)
    accuracy = float(modelResults_df.head(1)['Values'].values[0])

# Create files
with pd.ExcelWriter(linkedInModelResults_path) as writer:
    modelResults_df.to_excel(writer, sheet_name='Model Results', index=False)
    recsys_df.to_excel(writer, sheet_name='Model Outputs', index=False)
    recsysStats_df.to_excel(writer, sheet_name='Model Stats', index=False)
    recsysForValidation_df.to_excel(writer, sheet_name='Model Validation', index=False)
    training_df.to_excel(writer, sheet_name='Training Data', index=False)
    testing_df.to_excel(writer, sheet_name='Testing Data', index=False)
model.save(linkedInModel_path)

# TODO
# we talk about linkedin first for now
# what are the companies that are matched wrongly? any patterns? could be business user declaration and way of writing mismatches the business activity. maybe there's prob with source of truth. maybe sample issue skewed against certain ssic fields

Model Results:
Accuracy of Recommendation Model: 38.1%
Overall Similarity Score: 0.54 out of 1.00
Overall Stats (99 Divisions in Total):
mean    49.5
std     20.2
dtype: float64
Model Results:
Accuracy of Recommendation Model: 47.6%
Overall Similarity Score: 0.61 out of 1.00
Overall Stats (99 Divisions in Total):
mean    51.3
std     15.6
dtype: float64
Model Results:
Accuracy of Recommendation Model: 47.6%
Overall Similarity Score: 0.57 out of 1.00
Overall Stats (99 Divisions in Total):
mean    42.9
std     14.7
dtype: float64
Model Results:
Accuracy of Recommendation Model: 33.3%
Overall Similarity Score: 0.58 out of 1.00
Overall Stats (99 Divisions in Total):
mean    44.3
std     16.6
dtype: float64
Model Results:
Accuracy of Recommendation Model: 38.1%
Overall Similarity Score: 0.55 out of 1.00
Overall Stats (99 Divisions in Total):
mean    49.2
std     21.7
dtype: float64
Model Results:
Accuracy of Recommendation Model: 33.3%
Overall Similarity Score: 0.58 out of 1.00
Overall Stat

### Annual Reports

In [119]:
pdf = pdf[['PDF Name', 'UEN Number', 'Notes Page Content']]
pdf.columns = ['Company', 'UEN', 'Text Content']

In [120]:
pdf = pd.merge(pdf,ref[['ssic_code', 'UEN']], how = 'left', on = 'UEN')
pdf = pdf[['Company', 'ssic_code', 'Text Content']]


In [121]:
# Extracting only the name part into a new column
pdf['Company'] = pdf['Company'].apply(lambda x: ' '.join(x.split(' ')[:-1])[:-5].strip() if '.' in x else x[:-9].strip())
pdf['Company'] = pdf['Company'].apply(lambda x: x[:-1] if x[-1] == '.' else x)

In [122]:
accuracy = 0
for i in range(1,11):
# while accuracy < 0.55:
    recsys_df, recsysStats_df, recsysForValidation_df, training_df, testing_df, model, modelResults_df = recommendationModel(df = pdf, ssic = ssic,
                                                                                                    withSSIC = True, epochs = 40, vector_size = 64,
                                                                                                    top = 5, trainPercentage_0to1 = 0.7)
    accuracy = float(modelResults_df.head(1)['Values'].values[0])

# Create files
with pd.ExcelWriter(pdfModelResults_path) as writer:
    modelResults_df.to_excel(writer, sheet_name='Model Results', index=False)
    recsys_df.to_excel(writer, sheet_name='Model Outputs', index=False)
    recsysStats_df.to_excel(writer, sheet_name='Model Stats', index=False)
    recsysForValidation_df.to_excel(writer, sheet_name='Model Validation', index=False)
    training_df.to_excel(writer, sheet_name='Training Data', index=False)
    testing_df.to_excel(writer, sheet_name='Testing Data', index=False)
model.save(pdfModel_path)

recSSICDescription:
0    Manufacture of petroleum lubricating oil.This Sub-class includes manufacture of petroleum lubricating oil. Units in this Sub-class are primarily engaged in blending or compounding refined petroleum to make lubricating oils and greases and/or re-refining used petroleum lubricating oils.. Manufacture of petroleum lubricating oil.Lubricating oils, blending.
Name: Text Content, dtype: object
Error: index 0 is out of bounds for axis 0 with size 0
SSIC in Master List: 19202
recSSICDescription:
0    Building and repairing of ships, tankers and other ocean-going vessels (including conversion of ships into off-shore structures).This Sub-class includes the building and/or repairing of ships, except vessels for sports or recreation, and/or the construction of floating structures such as floating cranes and/or drilling platforms. This Sub-class also includes - manufacture of sections for ships and/or floating structures - building and/or repairing of: - commercial vessels:

### LinkedIn + PDF

In [123]:
combined_df = pd.merge(pdf, df, how = 'left', on = 'Company')
combined_df = combined_df[combined_df.ssic_code_x.notnull()]
combined_df['Text Content_y'] = combined_df['Text Content_y'].apply(lambda x: '' if pd.isnull(x) else x)
combined_df['Text Content'] = combined_df['Text Content_x'] + '.' + combined_df['Text Content_y']
combined_df.drop(columns = ['ssic_code_y', 'Text Content_x', 'Text Content_y'], inplace=True)
combined_df.rename(columns = {'ssic_code_x': 'ssic_code'}, inplace = True)

In [131]:
accuracy = 0
for i in range(1,11):
# while accuracy < 0.55:
    recsys_df, recsysStats_df, recsysForValidation_df, training_df, testing_df, model, modelResults_df = recommendationModel(df = combined_df, ssic = ssic,
                                                                                                    withSSIC = True, epochs = 40, vector_size = 64,
                                                                                                    top = 5, trainPercentage_0to1 = 0.7)
    accuracy = float(modelResults_df.head(1)['Values'].values[0])

# Create files
with pd.ExcelWriter(linkedInPdfModelResults_path) as writer:
    modelResults_df.to_excel(writer, sheet_name='Model Results', index=False)
    recsys_df.to_excel(writer, sheet_name='Model Outputs', index=False)
    recsysStats_df.to_excel(writer, sheet_name='Model Stats', index=False)
    recsysForValidation_df.to_excel(writer, sheet_name='Model Validation', index=False)
    training_df.to_excel(writer, sheet_name='Training Data', index=False)
    testing_df.to_excel(writer, sheet_name='Testing Data', index=False)
model.save(linkedInPdfModel_path)

actualSSICDescription:
Series([], Name: Text Content, dtype: object)
Error: index 0 is out of bounds for axis 0 with size 0
SSIC in Master List: 00nan
actualSSICDescription:
Series([], Name: Text Content, dtype: object)
Error: index 0 is out of bounds for axis 0 with size 0
SSIC in Master List: 00nan
actualSSICDescription:
Series([], Name: Text Content, dtype: object)
Error: index 0 is out of bounds for axis 0 with size 0
SSIC in Master List: 00nan
actualSSICDescription:
Series([], Name: Text Content, dtype: object)
Error: index 0 is out of bounds for axis 0 with size 0
SSIC in Master List: 00nan
actualSSICDescription:
Series([], Name: Text Content, dtype: object)
Error: index 0 is out of bounds for axis 0 with size 0
SSIC in Master List: 00nan
Model Results:
Accuracy of Recommendation Model: 38.5%
Overall Similarity Score: 0.55 out of 1.00
Overall Stats (99 Divisions in Total):
mean    50.4
std     19.1
dtype: float64
actualSSICDescription:
Series([], Name: Text Content, dtype: object